An implementation of encoded $\ket{A^2} = \ket{\sqrt{-iY}} = e^{-i \frac{\pi}{4}Y} \ket{+} = \ket{1}$ preparation in the Steane code using stim.

In particular, $A^2 = XH$, which will be used to implement the gate in stim.

Maybe easiest to just do $Y$ eigenstate, $\ket{+i}$?

In [2]:
from stim import Circuit
from noisy_circuits import noisy_steane_encoder,noisy_repetition_encoder,noisy_encoded_y
from decoders import steane_transversal_decoder,repetition_transversal_xdecoder

The logical $X$ outcome for the repetition-encoded ancilla block indicates which eigenstate the Hadamard test tells us it projected onto.

The logical $Y$ outcome for the Steane-encoded data block will indicate which eigenstate is observed.

If these two observables do not match, then there is a logical error on the output.

Below is a test of the circuit without errors, which should give us no errors and always result in selected trials.

In [3]:
def construct_noisy_circuit(perr:float) -> Circuit:
    r"""
    Method for producing a noisy circuit that in the absence of errors
    will initialize and measure the logical Pauli-Y eigenstate |+i>.
    In this particular circuit, the logical qubit is encoded in a Steane code
    while the logical ancilla is encoded into a seven-qubit repetition code.
    """

    noisy_circ = Circuit()

    steaneBlock = [0,1,2,3,4,5,6]
    repnBlock = [7,8,9,10,11,12,13]

    flag = True

    # initialize the Y-eigenstate: S^dagger|+>

    noisy_circ.append("RX",steaneBlock[0])
    noisy_circ.append("S_DAG",steaneBlock[0])
    
    noisy_circ.append("RX",repnBlock[0])

    noisy_circ.append("DEPOLARIZE1",[steaneBlock[0],repnBlock[0]],perr)

    noisy_circ = noisy_steane_encoder(noisy_circ,steaneBlock,perr)
    noisy_circ = noisy_repetition_encoder(noisy_circ,repnBlock,perr,True) # include flag qubit in encoder

    noisy_circ = noisy_encoded_y(noisy_circ,steaneBlock,repnBlock,perr)

    noisy_circ.append("MX",repnBlock[:-1]) # readout transversal logical-X
    noisy_circ.append("MY",steaneBlock) # readout transversal logical-Y
        
    return noisy_circ

In [4]:
def decode_sample_output(rec:list[bool]) -> tuple[bool,bool]:
    r"""
    A function that decodes a repetition code (xs) and steane code (yx) observables.
   
    """

    xs = rec[1:8]
    ys = rec[8:]

    mx = sum(xs)%2

    my = steane_transversal_decoder(ys)

    return mx,my

In [ ]:
error_rates = [0,
              1.0*10**(-5), 2.5*10**(-5),5.0*10**(-5),7.5*10**(-5),
              1.0*10**(-4), 2.5*10**(-4),5.0*10**(-4),7.5*10**(-4),
              1.0*10**(-3), 2.5*10**(-3),5.0*10**(-3),7.5*10**(-3),
              1.0*10**(-2)]

results = []

shots = 10_000_000

for perr in error_rates:
    selected = 0
    errors = 0
    
    noisy_circ = construct_noisy_circuit(perr)
    sampler = noisy_circ.compile_sampler()
    sample = sampler.sample(shots)

    for rec in sample:
        if not rec[0]: # post-select on False flag qubit
            mx,my = decode_sample_output(rec)

            if not mx: # post-select on expected observable for Y_L =+1
                selected +=1
                errors += not(mx==my)
        
    print(str(perr) + ', ' + str([selected/shots,errors/selected]))
    print()

0, [1.0, 0.0]

1e-05, [0.9993503, 4.352828032372632e-05]

2.5e-05, [0.9983614, 0.00010737594622548509]

5e-05, [0.9967139, 0.00022845071188432308]

7.500000000000001e-05, [0.9950673, 0.00034490129461595213]

0.0001, [0.9934573, 0.0004474273831396679]

0.00025, [0.9837322, 0.0011638330025183683]

0.0005, [0.9678256, 0.002347633705907345]

0.00075, [0.9523612, 0.00357700418706684]

0.001, [0.9373937, 0.004841082247512438]

0.0025, [0.8544172, 0.01316780607881021]

0.005, [0.7418412, 0.029681824088497646]

0.0075, [0.6537855, 0.04922027178638865]

0.01, [0.5848589, 0.0710043054829122]

